<a href="https://colab.research.google.com/github/neurawn/AnhITCHFeedHandler/blob/main/TinyImagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential, Model

import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive.mount( '/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cd /content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200 && python "resnet.py"


{'n03584254': 0, 'n02403003': 1, 'n02056570': 2, 'n02769748': 3, 'n01443537': 4, 'n02129165': 5, 'n02814533': 6, 'n04259630': 7, 'n01774750': 8, 'n02410509': 9}
X_train shape:  (5000, 32, 32, 3)
y_train shape:  (5000,)
x_train shape: (5000, 32, 32, 3)
5000 train samples
500 test samples
y_train shape: (5000,)
2023-01-08 00:07:28.945736: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
305 resnet, input shape after conv layer 2:  (None, 32, 32, 64) filter:  64
312 resnet, input shape:  (None, 32, 32, 16) filter:  64
305 resnet, input shape after conv layer 2:  (None, 32, 32, 64) filter:  64
305 resnet, input shape after conv layer 2:  (None, 32, 32, 64) filter:  64
305 resnet, input shape after conv layer 2:  (None, 32, 32, 64) filter:  64
305 resnet, input shape after conv layer 2:  (None, 32, 32, 64) filter:  64
305 resnet, input shape after conv l

In [ ]:
"""
THIS LOAD DATA IS STRICTLY FOR MODULE_V1
"""
import os
import pickle
import scipy.misc
import numpy as np
from PIL import Image

# from scipy.misc import imread

num_classes = 10
root_dir_path = "/content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/"
wnids_path = root_dir_path + "wnids"
words_path = root_dir_path + "words"

def load_tiny_imagenet(path, resize='False', dtype=np.float32):
    """
    Inputs:
    - path: String giving path to the directory to load.
    - dtype: numpy datatype used to load the data.

    Returns: A tuple of
    - class_names: A list where class_names[i] is a list of strings giving the
        WordNet names for class i in the loaded dataset.
    - X_train: (N_tr, 3, 64, 64) array of training images
    - y_train: (N_tr,) array of training labels
    - X_val: (N_val, 3, 64, 64) array of validation images
    - y_val: (N_val,) array of validation labels
    - X_test: (N_test, 3, 64, 64) array of testing images.
    - y_test: (N_test,) array of test labels; if test labels are not available
        (such as in student code) then y_test will be None.
    """
    # First load wnids
    wnids_file = wnids_path + str(num_classes) + '.txt'
    with open(wnids_file, 'r') as f:
        wnids = [x.strip() for x in f]

    # Map wnids to integer labels
    wnid_to_label = {wnid: i for i, wnid in enumerate(wnids)}
    print(wnid_to_label)

# 	# Use words.txt to get names for each class
    # words_file = words_path + str(num_classes) + '.txt'
    words_file = words_path + '.txt'
    with open(words_file, 'r') as f:
        wnid_to_words = dict(line.split('\t') for line in f)
        for wnid, words in wnid_to_words.items():
            wnid_to_words[wnid] = [w.strip() for w in words.split(',')]
    class_names = [wnid_to_words[wnid] for wnid in wnids]

    # Next load training data.
    X_train = []
    y_train = []
    for i, wnid in enumerate(wnids):
        # To figure out the filenames we need to open the boxes file
        boxes_file = os.path.join(path, 'train', wnid, '%s_boxes.txt' % wnid)
        with open(boxes_file, 'r') as f:
            filenames = [x.split('\t')[0] for x in f]
        num_images = len(filenames)

        # X_train_block = np.zeros((num_images, 64, 64, 3), dtype=dtype)	
        X_train_block = np.zeros((num_images, 32, 32, 3), dtype=dtype)
        y_train_block = wnid_to_label[wnid] * np.ones(num_images, dtype=np.int64)

        for j, image_path in enumerate(filenames):
            image_path = os.path.join(path, 'train', wnid, 'images', image_path)
            image = Image.open(image_path)
            # image = image.resize((32, 32), Image.LANCZOS)
            image_array = []
            if (image.mode == "RGB"):
                image_array = np.array(image, dtype = 'uint8')
            else: # Grayscale:
                image_array = np.stack((image,) * 3, axis = -1)
            
            X_train_block[j] = image_array
        X_train.append(X_train_block)
        y_train.append(y_train_block)

        print("finish loading ", image_path)
    
            
    # We need to concatenate all training data
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    # y_train = y_train.reshape([5000, 1])

    # Next load validation data
    with open(os.path.join(path, 'val', 'val_annotations.txt'), 'r') as f:
        image_files = []
        val_wnids = []
        for line in f:
            # Select only validation images in chosen wnids set
            if line.split()[1] in wnids:
                image_file, wnid = line.split('\t')[:2]
                image_files.append(image_file)
                val_wnids.append(wnid)

        num_val = len(image_files)
        
        # X_val = np.zeros((num_val, 64, 64, 3), dtype=dtype)
        X_val = np.zeros((num_val, 32, 32, 3), dtype=dtype)
        y_val = np.array([wnid_to_label[wnid] for wnid in val_wnids])
 
        for i, image_file in enumerate(image_files):
            image_file = os.path.join(path, 'val', 'images', image_file)
            image = Image.open(image_path)
            # image = image.resize((32, 32), Image.LANCZOS)
            image_array = []
            if (image.mode == "RGB"):
                image_array = np.array(image, dtype = 'uint8')
            else: # Grayscale:
                image_array = np.stack((image,) * 3, axis = -1)

            X_val[i] = image_array

    return class_names, X_train, y_train, X_val, y_val


In [ ]:
import numpy as np
from tensorflow.keras.layers import add, AveragePooling2D
from tensorflow.keras.models import load_model
class util:
    def __init__(self):
        self.convW =[]
        self.convB = []
        self.batchGamma = []
        self.batchBeta = []
        self.dense1W = []
        self.dense1B = []
        self.b1 = None
        self.b2 = None
        self.b3 = None
        self.D1 = None
        self.D2 = None
        self.D3 = None
        self.d1 = None
        self.d2 = None
        self.d3 = None
        self.first = True
        self.convWList = []
        self.convBList = []
        self.convMap = []
        self.X1proposed = None
        self.X1Previous = None
        self.first_use = True
               
    def getweights(self, nm):
        for i in range(0,len(nm.layers)):
            if("conv" in nm.layers[i].name):
                if (np.vstack([nm.layers[i].get_weights()[0]]).shape[0] == 3):
                    self.convW.append(np.vstack([nm.layers[i].get_weights()[0]]))
                    self.convB.append(nm.layers[i].get_weights()[1])
                # print ("shape: ", np.vstack([nm.layers[i].get_weights()[0]]).shape)
        for i in range(0,len(nm.layers)):
            if("batch" in nm.layers[i].name):
                self.batchGamma.append(nm.layers[i].get_weights()[0])
                self.batchBeta.append(nm.layers[i].get_weights()[1])
        for i in range(0,len(nm.layers)):
            if("dense" in nm.layers[i].name):
                self.dense1W=np.vstack([nm.layers[i].get_weights()[0]])
                self.dense1B=nm.layers[i].get_weights()[1]
        self.batchGamma.append(0)
        self.batchBeta.append(1)
        self.D2 = np.zeros_like(self.dense1W)
        self.d2 = np.zeros_like(self.dense1B)
        return self.convW, self.convB, self.batchGamma, self.batchBeta, self.dense1W, self.dense1B

    def softmax(self,x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0) # only difference

    def find(self,lst):
        return [i for i, x in enumerate(lst) if x<=0]
    def find2(self,lst):
        return [i for i, x in enumerate(lst) if x>0]

    def convStrictOperation(self, x, W, B, number, strides):
        convX= self.conv2d(x, W,  stride = (strides,strides)) + B
        if(self.first_use==False):
            mapTemp=self.convMap[number]
        else:
            mapTemp=[]
        #WTemp=[]
        #BTemp=[]
        convXtemp=convX.reshape(convX.shape[0]*convX.shape[1]*convX.shape[2]*convX.shape[3])
        zeroConvX=self.find(convXtemp)
        if(self.first_use==True):
            mapTemp=zeroConvX
        else:
            positiveConvX=self.find2(convXtemp)
            notPresentlist=list(set(positiveConvX).intersection(self.convMap[number]))
            for i in notPresentlist:
                indexItem=self.convMap[number].index(i)
                del self.convMap[number][indexItem] # removing inactive nodes in the layer.
        return convX, mapTemp, #WTemp, BTemp

    def denseOperation(self, x, W, B, D,d):
        x=np.dot(x,W)
        x=np.add(x, B)
        x[x<0]=0
        for i in range(x.shape[0]):
            if x[i] <= 0:
                D[:,i] = [x for x in D[:,i]]
            else:
                D[:,i] = W[:,i]
                d[i] = B[i]
        return x, D, d
    
    def copyWeightandBias(self,W1,b1,x,strides):
        #X = img.reshape((1,32,32,3)) 
        W1=np.vstack([W1])
        X1=self.conv2d(x, W1,  stride = (strides,strides)) + b1
        X21=X1.reshape(X1.shape[0]*X1.shape[1]*X1.shape[2]*X1.shape[3])
        convB=[]
        convW=[]
        for i in range(0,len(X21)):
            convW.append(W1)
            convB.append(b1)
        return convW, convB

    def resnet_layer_Predict(self,inputs, W, B, Gamma, Beta,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
        """2D Convolution-Batch Normalization-Activation stack builder
    
        Arguments:
            inputs (tensor): input tensor from input image or previous layer
            strides (int): Conv2D square stride dimensions
            activation (string): activation name
            batch_normalization (bool): whether to include batch normalization
            conv_first (bool): conv-bn-activation (True) or
                bn-activation-conv (False)
    
        Returns:
            x (tensor): tensor as input to the next layer
        """
        x = inputs
        if conv_first:
            #x = self.convPredict(x, W, B, strides)
            x = self.conv2d(inputs, W, stride = (strides,strides)) + B

            if batch_normalization:
                x = self.batchnorm_forward(x,Gamma, Beta)
            if activation is not None:
                x = self.relu(x)
        else:
            if batch_normalization:
                x = self.batchnorm_forward(x,Gamma, Beta)
            if activation is not None:
                x = self.relu(x)
            #x = self.convPredict(x, W, B, strides)
            x = self.conv2d(x, W, stride = (strides, strides)) + B
        return x

    def simplePredict(self,inputs, input_shape, convW, convB, GammaList, BetaList, denseW, denseB, num_classes=10,depth=0):
        """ResNet Version 1 Model builder [a]
    
        Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
        Last ReLU is after the shortcut connection.
        At the beginning of each stage, the feature map size is halved
        (downsampled) by a convolutional layer with strides=2, while 
        the number of filters is doubled. Within each stage, 
        the layers have the same number filters and the
        same number of filters.
        Features maps sizes:
        stage 0: 32x32, 16
        stage 1: 16x16, 32
        stage 2:  8x8,  64
        The Number of parameters is approx the same as Table 6 of [a]:
        ResNet20 0.27M
        ResNet32 0.46M
        ResNet44 0.66M
        ResNet56 0.85M
        ResNet110 1.7M
    
        Arguments:
            input_shape (tensor): shape of input image tensor
            depth (int): number of core convolutional layers
            num_classes (int): number of classes (CIFAR10 has 10)
    
        Returns:
            model (Model): Keras model instance
        """
        if (depth - 2) % 6 != 0:
            raise ValueError('depth should be 6n+2 (eg 20, 32, in [a])')
        # start model definition.
        num_filters = 16
        num_res_blocks = int((depth - 2) / 6)
        i=0
        j=0
        
        inputs  = inputs.reshape((1, inputs.shape[0], inputs.shape[1], inputs.shape[2]))
        x       = self.resnet_layer_Predict(inputs,convW[i], convB[i], GammaList[j], GammaList[j], batch_normalization=False)

        if self.first_use:
            convX, mapList = self.convStrictOperation(inputs, np.vstack([convW[i]]), convB[i], i, 1)
            self.convMap.append(mapList)
        else:
            convX, self.convMap[i] = self.convStrictOperation(inputs, np.vstack([convW[i]]), convB[i], i, 1)

        i = i + 1

        # instantiate the stack of residual units
        for stack in range(3):
            for res_block in range(num_res_blocks):
                strides = 1
                # first layer but not first stack
                if stack > 0 and res_block == 0:  
                    strides = 2  # downsample
                if self.first_use:
                    convY, mapList = self.convStrictOperation(x, np.vstack([convW[i]]), convB[i], i, strides)
                    self.convMap.append(mapList)
                else:
                    convY, self.convMap[i]=self.convStrictOperation(x, np.vstack([convW[i]]), convB[i], i, strides)
                y = self.resnet_layer_Predict(x, convW[i], convB[i], GammaList[j], GammaList[j], strides=strides, batch_normalization=False)

                i=i+1
                
                if self.first_use:
                    convY, mapList = self.convStrictOperation(y, np.vstack([convW[i]]), convB[i], i, strides)
                    self.convMap.append(mapList)
                else:
                    convY, self.convMap[i] = self.convStrictOperation(y, np.vstack([convW[i]]), convB[i], i, strides)
                y = self.resnet_layer_Predict(y, convW[i], convB[i], GammaList[j], GammaList[j], activation=None, batch_normalization=False)
                i=i+1
                #j=j+1
                # first layer but not first stack
                if stack > 0 and res_block == 0:
                    if self.first_use:
                        convX, mapList = self.convStrictOperation(x, np.vstack([convW[i]]), convB[i], i, strides)
                        self.convMap.append(mapList)
                    else:
                        convX, self.convMap[i]=self.convStrictOperation(x, np.vstack([convW[i]]), convB[i], i, strides)
                    x = self.resnet_layer_Predict(x, convW[i], convB[i], GammaList[j], GammaList[j], 
                                     strides=strides,
                                     activation=None,
                                     batch_normalization=False)
                    i=i+1
                x = add([y, x])
                x = self.relu(x)
    
        # add classifier on top.
        # v1 does not use BN after last shortcut connection-ReLU
        x=AveragePooling2D(pool_size=8)(x).numpy()
        y = x.reshape(x.shape[0] * x.shape[1] * x.shape[2] * x.shape[3])
        denseX, self.D2, self.d2 = self.denseOperation(y, denseW, denseB, self.D2, self.d2)
        x = np.dot(x,denseW)
        x = np.add(x,denseB)
        self.X1proposed=x
        x = self.softmax(x.reshape(x.shape[3]))
        self.first_use=False
        # instantiate model.
        #model = Model(inputs=inputs, outputs=outputs)
        return self.convMap, self.D2, self.d2                           

    def relu(self,X):
        return np.maximum(X, 0)

    def batchnorm_forward(self,X, gamma = 1, beta = 0):
        mu = np.mean(X, axis=0)
        var = np.var(X, axis=0)
    
        X_norm = (X - mu) / np.sqrt(var + 1e-8)
        out = gamma * X_norm + beta
        
        return out

    def conv2d(self, x, w, pad='SAME', stride=(1, 1)):
        """2D convolution (technically speaking, correlation).
    
        Args:
            x: [N, H, W, C]
            w: [I, J, C, K]
            pad: [PH, PW]
            stride: [SH, SW]
    
        Returns:
            y: [N, H', W', K]
        """
        ksize = w.shape[:2]
        x = self.extract_sliding_windows(x, ksize, pad, stride)
        
        ws = w.shape
        w = w.reshape([ws[0] * ws[1] * ws[2], ws[3]])
        xs = x.shape
        x = x.reshape([xs[0] * xs[1] * xs[2], -1])
        y = x.dot(w)
        y = y.reshape([xs[0], xs[1], xs[2], -1])
        return y

    def extract_sliding_windows(self,x, ksize, pad, stride, floor_first=True):
        """Converts a tensor to sliding windows.
    
        Args:
            x: [N, H, W, C]
            k: [KH, KW]
            pad: [PH, PW]
            stride: [SH, SW]
    
        Returns:
            y: [N, (H-KH+PH+1)/SH, (W-KW+PW+1)/SW, KH * KW, C]
        """
        n = x.shape[0]
        h = x.shape[1]
        w = x.shape[2]
        c = x.shape[3]
        kh = ksize[0]
        kw = ksize[1]
        sh = stride[0]
        sw = stride[1]
    
        h2 = int(self.calc_size(h, kh, pad, sh))
        w2 = int(self.calc_size(w, kw, pad, sw))
        ph = int(self.calc_pad(pad, h, h2, sh, kh))
        pw = int(self.calc_pad(pad, w, w2, sw, kw))
    
        ph0 = int(np.floor(ph / 2))
        ph1 = int(np.ceil(ph / 2))
        pw0 = int(np.floor(pw / 2))
        pw1 = int(np.ceil(pw / 2))
        if(ph0==-1):
            ph0=0
        if(pw0==-1):
            pw0=0
        if floor_first:
            pph = (ph0, ph1)
            ppw = (pw0, pw1)
        else:
            pph = (ph1, ph0)
            ppw = (pw1, pw0)
        x = np.pad(
            x, ((0, 0), pph, ppw, (0, 0)),
            mode='constant',
            constant_values=(0.0, ))
    
        # The following code extracts window without copying the data:
        # y = np.zeros([n, h2, w2, kh, kw, c])
        # for ii in range(h2):
        #     for jj in range(w2):
        #         xx = ii * sh
        #         yy = jj * sw
        #         y[:, ii, jj, :, :, :] = x[:, xx:xx + kh, yy:yy + kw, :]
        x_sn, x_sh, x_sw, x_sc = x.strides
        y_strides = (x_sn, sh * x_sh, sw * x_sw, x_sh, x_sw, x_sc)
        y = np.ndarray((n, h2, w2, kh, kw, c),
                       dtype=x.dtype,
                       buffer=x.data,
                       offset=self.array_offset(x),
                       strides=y_strides)
        return y
    def softmax(self,x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
    def calc_size(self, h, kh, pad, sh):
        """Calculate output image size on one dimension.
    
        Args:
            h: input image size.
            kh: kernel size.
            pad: padding strategy.
            sh: stride.
    
        Returns:
            s: output size.
        """
    
        if pad == 'VALID':
            return np.ceil((h - kh + 1) / sh)
        elif pad == 'SAME':
            return np.ceil(h / sh)
        else:
            return int(np.ceil((h - kh + pad + 1) / sh))
    def calc_pad(self,pad, in_siz, out_siz, stride, ksize):
        """Calculate padding width.
    
        Args:
            pad: padding method, "SAME", "VALID", or manually speicified.
            ksize: kernel size [I, J].
    
        Returns:
            pad_: Actual padding width.
        """
        if pad == 'SAME':
            return (out_siz - 1) * stride + ksize - in_siz
        elif pad == 'VALID':
            return 0
        else:
            return pad
    
    def array_offset(self,x):
        """Get offset of array data from base data in bytes."""
        if x.base is None:
            return 0
    
        base_start = x.base.__array_interface__['data'][0]
        start = x.__array_interface__['data'][0]
        return start - base_start 

In [ ]:
# from utils.cifarUtil import cifarUtilClass
from tensorflow.keras.models import load_model
from datetime import datetime
# from load_data import load_tiny_imagenet
import numpy as np
import pickle
import tensorflow as tf
tf.keras.backend.set_floatx('float64')
# In[]: Data Acquisition and Model Load
path = "/content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/"
model=load_model('/content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/resnet56_v1Full_test.h5')
class_name, xT, yT, xt, yt = load_tiny_imagenet(path)
print ("yT shape:", yT.shape, "yt shape:", yt.shape)
# In[]: Module Creation
test=[]
accuracyCheck=[]
nonDigit=[]
resNet=57
labels = [0,1,2,3,4,5,6,7,8,9]
input_shape = xt.shape[1:]
print ("input shape: ", input_shape)
print("Start Time:"+datetime.now().strftime("%H:%M:%S"))
for j in labels:
    print("#Module "+str(j)+" in progress....")
    digit  = []
    slc = util()
    convW, convB, batchGamma, batchBeta, dense1W, dense1B=slc.getweights(model)
    for i in range(0,len(yt)):
        if yT[i] == j and model.predict(xT[i:i+1], verbose = 0)[0][j] >.9:
            digit.append(xT[i])

    digit = digit[0:10]
    mc = 0
    for x in digit:
        maplist, D2, d2 = slc.simplePredict(x, input_shape, convW, convB, batchGamma, batchBeta, dense1W, dense1B, depth=56)
        mc = mc+1
        # print('+ve Here, mc: ', mc)
        if np.count_nonzero(slc.D2) < 45:
            #print("Breaking at mc ", mc,np.count_nonzero(slc.D2))
            slc.first = True

    digit=[]

    for k in labels:
        if(k!=j):
            temp=xT[yT==k]
            temp=temp[0:10]
            for u in temp:
                digit.append(u)

    np.random.shuffle(digit)
    mc = 0
    for x in digit:
        maplist, D2, d2 = slc.simplePredict(x, input_shape, convW, convB, batchGamma, batchBeta, dense1W, dense1B, depth=56)
        mc = mc+1
        # print('-ve Here, mc: ', mc)
        if np.count_nonzero(slc.D3) < 45:
            #print("Breaking at mc ", mc,np.count_nonzero(slc.D2))
            slc.first = True
    #In[]: Verifiy
    for i in range(0, resNet):
        with open("/content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/modeltext/Module"+str(j)+"-"+str(i)+"Map.txt", "w+") as fp:   #Pickling
            print ("j: " + str(j) + "maplist[" + str(i)+ "]", maplist[i])
            # fp.write("\n".join(str(item) for item in maplist[i]))
    # with open("/content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/modeltext/Module"+str(j)+"DenseW.txt", "w+") as fp:   #Pickling
        print("j: " + str(j) + "D2: ", D2)
        # fp.write(D2)
    # with open("/content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/modeltext/Module"+str(j)+"DenseB.txt", "w+") as fp:   #Pickling
        print("j: " + str(j) + "d2: ", d2)
        # fp.write(d2)


{'n03584254': 0, 'n02403003': 1, 'n02056570': 2, 'n02769748': 3, 'n01443537': 4, 'n02129165': 5, 'n02814533': 6, 'n04259630': 7, 'n01774750': 8, 'n02410509': 9}
finish loading  /content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/train/n03584254/images/n03584254_499.JPEG
finish loading  /content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/train/n02403003/images/n02403003_499.JPEG
finish loading  /content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/train/n02056570/images/n02056570_499.JPEG
finish loading  /content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/train/n02769748/images/n02769748_499.JPEG
finish loading  /content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/train/n01443537/images/n01443537_499.JPEG
finish loading  /content/gdrive/MyDrive/ColabNotebooks/Decomposer_2/ImageNet/tiny-imagenet-200/train/n02129165/images/n02129165_499.JPEG
finish loading  /

ValueError: ignored